# Weaving

Two types of knots: 
* weft-facing
* warp facing

Each cell contains single knot.

Any neighbour combinations are possible.


In [1]:
from random import random
import numpy as np
import sympy as sp
from ipywidgets import widgets
import k3d

from myutils import *
from mycurves import *

t, s, u, v, r, a, b, c, d = sp.symbols("t, s, u, v, r, a, b, c, d")
P, P0, P1, P2, P3 = vec.symbols('P P0 P1 P2 P3')

In [2]:
from matplotlib.cm import get_cmap
palette = hexpalette(get_cmap('tab10'))

# Topology

* single cross point in the middle of a cell
* pattern determines elevation of weft yarn to be ±1
* elevation of warp yarn is just opposite

Anchor points:
* $A_{wft}$ — center of weft spline
* $A_{wrp}$ — center of warp spline

Connectivity:
* $A^W_{wft}$ — weft west neighbour
* $A^E_{wft}$ — weft east neighbour
* $A^S_{wrp}$ — warp south neighbour
* $A^N_{wrp}$ — warp north neighbour


In [16]:
pattern = np.array([
    [+1, +1, -1, +1, +1],
    [-1, +1, +1, -1, +1],
    [+1, -1, +1, +1, -1],
    [+1, +1, -1, +1, +1],    
    [-1, +1, +1, -1, +1],    
])

In [20]:
plot = k3d.plot()

for u in range(0, len(pattern)):
    row = pattern[u]
    points = [(0.5 + v, 0.5 + u, 0.5 * row[v]) for v in range(len(row))]
    plot += k3d.points(points, color=0xff0000, point_size=.1, opacity=0.5)
    plot += k3d.line(points, color=0xff0000, line_width=0.1, opacity=0.5)
plot += k3d.text2d("weft", (0, 0.1), color=0xff0000)

pattern_t = pattern.T
for v in range(0, len(pattern_t)):
    col = pattern_t[v]
    points = [(0.5 + v, 0.5 + u, -0.5 * col[u]) for u in range(len(col))]
    plot += k3d.points(points, color=0x00ff00, point_size=.1, opacity=0.5)
    plot += k3d.line(points, color=0x00ff00, line_width=0.1, opacity=0.5)
plot += k3d.text2d("warp", (0, 0.2), color=0x00ff00)

plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

# Geometry

Parameters:
* $R_{wrp}$ — warp radius
* $R_{wft}$ — weft radius
* $Kq_{wrp}$ — warp squeezing factor
* $Kq_{wft}$ — weft squeezing factor
* $Ks_{wrp}$ — warp stiffness, restrict bending

Constraints:
* warp and weft fully connected at knots, total thickness = sum of diameters
* curvature at contact point is enough to embrace opposite yarn → fuckery

Anchors points:
* $A^W_{wft}$ — west neighbour
* $A_{wft}$ — weft center
* $A^E_{wft}$ — east neighbour
* $A^S_{wrp}s$ — south neighbour
* $A_{wrp}$ — warp center
* $A^N_{wrp}$ — north neighbour

Spline points:
* $P^W_{e}, P_{w}, P_{e}, P^E_{w}$ - weft spline
* $P^S_{n}, P_{s}, P_{n}, P^N_{s}$ - warp spline

In [3]:
north = vector(0, 1, 0)
south = vector(0,-1, 0)
west = vector(-1, 0, 0)
east = vector(1, 0, 0)
mirror = lambda v: vector(1-v[0], v[1], v[2])  # E <-> W
zed = lambda v, z: vector(v[0], v[1], z) 
scaled = lambda v, scale: vector(v[0]*scale[0], v[1]*scale[1], v[2])

## elevation fix

Matching z coord with opposite yarn height

* $H_{wrp} = R_{wrp} (1 - Kq_{wrp})$ — warp bump height
* $H_{wft} = R_{wft} (1 - Kq_{wft})$ — weft bump height
* $H = H_{wrp} + H_{wft}$ — total half-thickness
* $\alpha_{wrp} \in [0, H]$ — warp waves half-amplitude
* $\alpha_{wft} \in [0, H]$ — weft waves half-amplitude

$\alpha_{wrp} + H_{wrp} + \alpha_{wft} + H_{wft} = 2 H$ — total thickness equation

$\alpha_{wrp} + \alpha_{wft} = H$

$\alpha_{wrp} = H(1 - Ks)$ — stifness tuning

$\alpha_{wft} = H - \alpha_{wrp}$

Result:

$A_{wft} = A_{cross} + e_z \alpha_{wft}$

$A_{wrp} = A_{cross} - e_z \alpha_{wrp}$







## curvature fix

Edge points are aligned with knot point along XY to provide smooth tangent.

* $D_{wft} = e_x 0.5 K_k $ — curvature fix
* $D_{wrp} = e_y 0.5 K_k $ — curvature fix
* $P_{w} = A^c_{wft} - D_{wft}$
* $P_{e} = A^c_{wft} + D_{wft}$
* $P_{s} = A^c_{wrp} - D_{wrp}$
* $P_{n} = A^c_{wrp} + D_{wrp}$

# Plot

Cell-based plotting to use shading calc pipeline.

Geometry is calculated in cell local coords.

In [107]:
def topology(pattern, x, y):
    """grid -> pattern -> topology"""
    return {
        'c': vector( 0.5, 0.5, pattern[y][x]),
        'e': vector( 1.5, 0.5, pattern[y][x+1]),
        'w': vector(-0.5, 0.5, pattern[y][x-1]),
        's': vector( 0.5,-0.5, pattern[y-1][x]),
        'n': vector( 0.5, 1.5, pattern[y+1][x])
    }

def geometry(topo, params):
    """topology -> geometry"""
    geom = lambda v, a: vector(v[0], v[1],  a * v[2])
    
    # amplitudes
    h_wrp = params['r_wrp'] * (1 - params['sq_wrp'])
    h_wft = params['r_wft'] * (1 - params['sq_wft'])
    h = h_wft + h_wrp
    a_wrp = h * (1 - params['stf_wrp'])
    a_wft = h - a_wrp

    w_wft = geom(topo['w'], a_wft)
    c_wft = geom(topo['c'], a_wft)
    e_wft = geom(topo['e'], a_wft)
    s_wrp = geom(topo['s'], -a_wrp)
    c_wrp = geom(topo['c'], -a_wrp)
    n_wrp = geom(topo['n'], -a_wrp)
    
    krv_wft = vector(0.5, 0, 0) * params['krv_wft']
    krv_wrp = vector(0, 0.5, 0) * params['krv_wrp'] 
    
    points = {
        'a': {
            'c': topo['c']
        },
        'wft': [
            w_wft + krv_wft,
            c_wft - krv_wft,
            c_wft + krv_wft,
            e_wft - krv_wft,
        ],
        'wrp': [
             s_wrp + krv_wrp,
             c_wrp - krv_wrp,            
             c_wrp + krv_wrp,            
             n_wrp - krv_wrp,
        ]
    }
    
    return points

In [4]:
plot = k3d.plot(camera_auto_fit=False, grid_auto_fit=False, grid=(1, 1, -1, 4, 4, 1))

In [5]:
controls = {
    'render': widgets.Dropdown(description='Render:',
            options=[('Points', 'C'), ('Splines', 'S'), ('Pipes', 'P')],
            value='S'),
    'sx': widgets.FloatSlider(description='scale x', min=0, max=2, value=1.0, continuous_update=False),
    'sy': widgets.FloatSlider(description='scale y', min=0, max=2, value=1.0, continuous_update=False),
    'r_wrp': widgets.FloatSlider(description='warp r', min=0, max=0.5, value=0.25, continuous_update=False),
    'sq_wrp': widgets.FloatSlider(description='warp sq', min=0, max=1.0, value=0, continuous_update=False),
    'krv_wrp': widgets.FloatSlider(description='warp curv', min=0, max=1.0, value=0.5, continuous_update=False),
    'r_wft': widgets.FloatSlider(description='weft r', min=0, max=0.5, value=0.25, continuous_update=False),
    'sq_wft': widgets.FloatSlider(description='weft sq', min=0, max=1.0, value=0, continuous_update=False),
    'krv_wft': widgets.FloatSlider(description='weft curv', min=0, max=1.0, value=0.5, continuous_update=False),
    'stf_wrp': widgets.FloatSlider(description='stiffness', min=0, max=1.0, value=0.5, continuous_update=False),    
}

chklay = widgets.Layout(width="24px")
chk = lambda v: widgets.Checkbox(value=bool(v), indent=False, layout=chklay)
brd = widgets.VBox([
    widgets.HBox([chk(1), chk(0), chk(1), chk(0), chk(1)]),
    widgets.HBox([chk(0), chk(1), chk(0), chk(1), chk(0)]),
    widgets.HBox([chk(1), chk(0), chk(1), chk(0), chk(1)]),
    widgets.HBox([chk(0), chk(1), chk(0), chk(1), chk(0)]),
    widgets.HBox([chk(1), chk(0), chk(1), chk(0), chk(1)]),
])

def read_pattern():
    return [
        [ 1 if box.value else -1
            for box in hbox.children]
        for hbox in brd.children
    ]
    

In [6]:
widgets.GridBox([
    widgets.VBox([widgets.Label("pattern"), brd]),
    widgets.VBox(list(controls.values())),
    plot], 
    layout=widgets.Layout(grid_template_columns="auto auto auto"))

GridBox(children=(VBox(children=(Label(value='pattern'), VBox(children=(HBox(children=(Checkbox(value=True, in…

In [27]:
def clear():
    global plot
    objects = list(plot.objects)
    for obj in objects:
        plot -= obj

def draw(obj):
    global plot
    plot += obj

def project(x0, y0, sx, sy):
    return lambda v: scaled(vector(x0, y0, 0) + v,  (sx, sy))
    
def draw_points(x, y, params): 
    proj = project(x, y, params['sx'], params['sy'])
    topo = topology(read_pattern(), x, y)
    geom = geometry(topo, params)    
    
    ancpoints = [proj(geom['a']['c'])]
    draw(k3d.points(ancpoints, point_size=0.1, color=0x000000, shader="3d", opacity=0.5))
    
    wftpoints = [proj(v) for v in geom['wft']]
    draw(k3d.points(wftpoints, point_size=0.05, color=0xff0000, opacity=0.5, shader="3d"))
    draw(k3d.line(wftpoints, color=0xff0000, opacity=0.5, shader="3d"))

    wrppoints = [proj(v) for v in geom['wrp']]
    draw(k3d.points(wrppoints, point_size=0.05, color=0x00ff00, opacity=0.5, shader="3d"))
    draw(k3d.line(wrppoints, color=0x00ff00, opacity=0.5, shader="3d"))
    
def draw_splines(x, y, params): 
    proj = project(x, y, params['sx'], params['sy'])
    topo = topology(read_pattern(), x, y)
    geom = geometry(topo, params)    
    
    wftpoints = [proj(v) for v in geom['wft']]
    wftspline = eval_spline(B2, u, wftpoints)    
    draw(k3d.line(wftspline, line_width=0.1, color=0xff0000, opacity=0.5))

    wrppoints = [proj(v) for v in geom['wrp']]
    wrpspline = eval_spline(B2, u, wrppoints)    
    draw(k3d.line(wrpspline, line_width=0.1, color=0x00ff00, opacity=0.5))

def draw_pipes(x, y, params):
    proj = project(x, y, params['sx'], params['sy'])
    topo = topology(read_pattern(), x, y)
    geom = geometry(topo, params)    
    
    r_a_wrp = params['r_wrp']
    r_b_wrp = r_a_wrp * (1 - params['sq_wrp'])
    r_a_wft = params['r_wft']
    r_b_wft = r_a_wrp * (1 - params['sq_wft'])        
    
    Qc_wft = Ellipse(v, r_a_wft, r_b_wft)
    Qc_wrp = Ellipse(v, r_a_wrp, r_b_wrp)
    
    def draw_pipe(splpoints, color):
        for ids in iter_slices(3, len(splpoints)):
            Qa = Spline(u, B2, splpoints[ids])
            S = Pipe((u, v), Qa, Qc_wrp)
            pipe_pnts, pipe_tris, _ = eval_pipe(S, (u, v), 6, 12)
            draw(k3d.mesh(pipe_pnts, pipe_tris, color=color, flat_shading=False))
        
    wftpoints = [proj(v) for v in geom['wft']]
    draw_pipe(wftpoints, 0xff8080)
    wrppoints = [proj(v) for v in geom['wrp']]
    draw_pipe(wrppoints, 0x80ff80)
    
def redraw(*args):
    params = {k: w.value for k, w in controls.items()}
    clear()
    plot.grid = [0, 0, -1, 5*params['sx'], 5*params['sy'], 1]
    
    for y in range(1, 4):
        for x in range(1, 4):            
            if params['render'] == 'S':
                draw_splines(x, y, params)
            elif params['render'] == 'P':
                draw_pipes(x, y, params)
            else:
                draw_points(x, y, params)                

In [28]:
clear()

In [29]:
redraw()

In [11]:
for w in controls.values():
    w.observe(redraw, 'value')
    
for r in brd.children:
    for w in r.children:
        w.observe(redraw, 'value')